In [1]:
# -*- coding:utf-8 -*-


#######  尝试骚操作，单独针对这个表
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier, Ridge, \
    PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import LinearSVC, LinearSVR

train = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_train.tsv', sep='\t', header=None)
test = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_test.tsv', sep='\t', header=None)
test_id = test[0]
def get_label(row):
    return row[2]
train['label'] = train.apply(lambda row:get_label(row), axis=1)
data_all = pd.concat([train, test], axis=0)
data_all = data_all.rename({0:'id'}, axis=1)
del data_all[1],data_all[2]

deviceid_packages = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_packages.tsv', sep='\t', header=None)
deviceid_packages = deviceid_packages.rename({0: 'id', 1: 'packages_names'}, axis=1)
package_label = pd.read_csv('/Users/xuelili/eguan data/Demo/package_label.tsv', sep='\t', header=None)
package_label = package_label.rename({0:'packages_name', 1:'packages_type'},axis=1)
dict_label = dict(zip(list(package_label['packages_name']), list(package_label['packages_type'])))

data_all = pd.merge(data_all, deviceid_packages, on='id', how='left')

feature = pd.DataFrame()

import numpy as np

# app个数
# 毒特征？
# feature['app_count'] = data_all['packages_names'].apply(lambda row: len(str(row).split(',')))

# 对此数据做countvector,和tfidfvector,并在一起跑几个学习模型
# 引申出来的count和tfidf，跑基本机器学习分类模型
data_all['package_str'] = data_all['packages_names'].apply(lambda row: str(row).replace(',', ' '))
def get_more_information(row):
    result = ' '
    start = True
    row_list = row.split(',')
    for i in row_list:
        try:
            if start:
                result = dict_label[i]
                start = False
            else:
                result = result + ' ' + dict_label[i]
        except KeyError:
            pass
    return result
data_all['package_str_more_information'] = data_all['packages_names'].apply(lambda row: get_more_information(str(row)))

print(data_all)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

count_vec = CountVectorizer()
count_csr_basic = count_vec.fit_transform(data_all['package_str'])
tfidf_vec = TfidfVectorizer()
tfidf_vec_basic = tfidf_vec.fit_transform(data_all['package_str'])

count_vec = CountVectorizer()
count_csr_more = count_vec.fit_transform(data_all['package_str_more_information'])

tfidf_vec = TfidfVectorizer()
tfidf_vec_more = tfidf_vec.fit_transform(data_all['package_str_more_information'])

data_feature = scipy.sparse.csr_matrix(scipy.sparse.hstack([count_csr_basic, tfidf_vec_basic,
                     count_csr_more, tfidf_vec_more]))

train_feature = data_feature[:len(train)]
score = train['label']
test_feature = data_feature[len(train):]
number = len(np.unique(score))

X = train_feature
test = test_feature
y = score

n_flods = 5
kf = KFold(n_splits=n_flods,shuffle=True,random_state=1017)
kf = kf.split(X)

def xx_mse_s(y_true,y_pre):
    y_true = y_true
    y_pre = pd.DataFrame({'res': list(y_pre)})
    return mean_squared_error(y_true,y_pre['res'].values)

######################## ridge reg #########################3
cv_pred = []
xx_mse = []
stack = np.zeros((len(y),1))
stack_te = np.zeros((len(test_id),1))
model_1 = Ridge(solver='auto', fit_intercept=True, alpha=0.4, max_iter=250, normalize=False, tol=0.01,random_state=1017)
for i ,(train_fold,test_fold) in enumerate(kf):
    X_train, X_validate, label_train, label_validate = X[train_fold, :], X[test_fold, :], y[train_fold], y[test_fold]
    model_1.fit(X_train, label_train)
    val_ = model_1.predict(X=X_validate)
    stack[test_fold] = np.array(val_).reshape(len(val_),1)
    print(xx_mse_s(label_validate, val_))
    cv_pred.append(model_1.predict(test))
    xx_mse.append(xx_mse_s(label_validate, val_))
import numpy as np
print('xx_result',np.mean(xx_mse))
s = 0
for i in cv_pred:
    s = s+i
s = s/n_flods
print(stack)
print(s)
df_stack1 = pd.DataFrame(stack)
df_stack2 = pd.DataFrame(s)
df_stack = pd.concat([df_stack1,df_stack2
                ], axis=0)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_ling_reg.csv', encoding='utf8', index=None)

######################## par reg #########################
kf = KFold(n_splits=n_flods,shuffle=True,random_state=1017)
kf = kf.split(X)
cv_pred = []
xx_mse = []
stack = np.zeros((len(y),1))
model_1 = PassiveAggressiveRegressor(fit_intercept=True, max_iter=280, tol=0.01,random_state=1017)
for i ,(train_fold,test_fold) in enumerate(kf):
    X_train, X_validate, label_train, label_validate = X[train_fold, :], X[test_fold, :], y[train_fold], y[test_fold]
    model_1.fit(X_train, label_train)
    val_ = model_1.predict(X=X_validate)
    stack[test_fold] = np.array(val_).reshape(len(val_),1)
    print(xx_mse_s(label_validate, val_))
    cv_pred.append(model_1.predict(test))
    xx_mse.append(xx_mse_s(label_validate, val_))
import numpy as np
print('xx_result',np.mean(xx_mse))
s = 0
for i in cv_pred:
    s = s+i
s = s/n_flods
print(stack)
print(s)
df_stack1 = pd.DataFrame(stack)
df_stack2 = pd.DataFrame(s)
df_stack = pd.concat([df_stack1,df_stack2
                ], axis=0)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_par_reg.csv', encoding='utf8', index=None)

######################## svr reg #########################
kf = KFold(n_splits=n_flods,shuffle=True,random_state=1017)
kf = kf.split(X)
cv_pred = []
xx_mse = []
stack = np.zeros((len(y),1))
model_1 = LinearSVR(random_state=1017)
for i ,(train_fold,test_fold) in enumerate(kf):
    X_train, X_validate, label_train, label_validate = X[train_fold, :], X[test_fold, :], y[train_fold], y[test_fold]
    model_1.fit(X_train, label_train)
    val_ = model_1.predict(X=X_validate)
    stack[test_fold] = np.array(val_).reshape(len(val_),1)
    print(xx_mse_s(label_validate, val_))
    cv_pred.append(model_1.predict(test))
    xx_mse.append(xx_mse_s(label_validate, val_))
import numpy as np
print('xx_result',np.mean(xx_mse))
s = 0
for i in cv_pred:
    s = s+i
s = s/n_flods
print(stack)
print(s)
df_stack1 = pd.DataFrame(stack)
df_stack2 = pd.DataFrame(s)
df_stack = pd.concat([df_stack1,df_stack2
                ], axis=0)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_svr_reg.csv', encoding='utf8', index=None)



/Users/xuelili/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


                                     id  label  \
0      bd86d59afa24a839ce6029d718accb19    3.0   
1      e7d158c9a8262a35c9cc630a15a9103e    5.0   
2      97abdc3828448b5acc7428dd307bc635    5.0   
3      e4dbdbf07c9cff03d79f4872e65742b4    4.0   
4      6bd4537b2970c5c6ab765c1860b88aa5    3.0   
5      6cde2a7fba98a39c2fe69b52a5ffa8c4    5.0   
6      7e7e746f7fa507f102d8edb8fe7bcdb8    3.0   
7      a4c81166ac854f229791ba483399b301    4.0   
8      3864b53a73faa09cb0f83878823a1092    3.0   
9      abb95da7bd0c87213bb3da80bd18b3fb    5.0   
10     f8b1e3cc200ebb7cdfb5939429950fe1    7.0   
11     f9e15046c20c16e19264585d20f19f33    5.0   
12     947866a16a3361a54c42843ec8376042    5.0   
13     bca3c313ac1c90818f2761ec854eada8    6.0   
14     27070a8df98df8aa6c4e58707a1590f4    6.0   
15     1bdab3a0f66515900f3e858c4bc7c7a4    7.0   
16     63e579bfe2febf44b1f6cbe5cf4a0130    4.0   
17     808ba0f35e382ed557b138e21e5b6af0    6.0   
18     451cfe472891e82634272af946b92b76    6.0   


4.7031167791130075
4.75929491092615
4.738455474976471
4.765650501598204
4.724234807928267
xx_result 4.73815049490842
[[7.47660924]
 [7.38663393]
 [6.48724506]
 ...
 [5.68230705]
 [6.41367373]
 [6.18300597]]
[5.76726193 5.84300458 7.21033299 ... 7.35982753 4.63115596 6.82803545]
11.032941361510561
9.045485615328971
8.872624413962301
12.077334989769867
9.038047343572728
xx_result 10.013286744828886
[[ 6.39371717]
 [ 6.98101089]
 [ 9.6672826 ]
 ...
 [ 3.16983756]
 [ 6.5767296 ]
 [10.15079892]]
[6.06154688 4.77839582 6.08714379 ... 8.01474988 5.95977711 4.9558102 ]
5.180392361003141
5.290619635769758
5.228875731007849
5.307509277653756
5.239147809744366
xx_result 5.249308963035775
[[7.13286571]
 [5.62840712]
 [6.80915753]
 ...
 [5.29379454]
 [6.60798206]
 [6.17137401]]
[5.57567183 6.03239791 7.11638728 ... 8.65360177 5.16615901 7.06314447]


In [6]:
# -*- coding:utf-8 -*-

import pandas as pd
import numpy as np
from sklearn import preprocessing

train = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_train.tsv', sep='\t', header=None)
test = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_test.tsv', sep='\t', header=None)

data_all = pd.concat([train, test], axis=0)
data_all = data_all.rename({0:'id'}, axis=1)
del data_all[1],data_all[2]
deviced_brand = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_brand.tsv', sep='\t', header=None)
deviced_brand = deviced_brand.rename({0: 'id'}, axis=1)
data_all = pd.merge(data_all, deviced_brand, on='id', how='left')
print(data_all)
# 直接做类别编码特征
#from sklearn.preprocessing import LabelEncoder（没有该句）

feature = pd.DataFrame()
label_encoder = preprocessing.LabelEncoder()
feature['phone_type'] = label_encoder.fit_transform(list(data_all[1]))   #没有list报错
feature['phone_type_detail'] = label_encoder.fit_transform(list(data_all[2]))  #没有list报错
feature.to_csv('/Users/xuelili/eguan data/feature1/deviceid_brand_feature.csv', index=False)

                                     id           1  \
0      bd86d59afa24a839ce6029d718accb19        vivo   
1      e7d158c9a8262a35c9cc630a15a9103e      Huawei   
2      97abdc3828448b5acc7428dd307bc635     samsung   
3      e4dbdbf07c9cff03d79f4872e65742b4      Huawei   
4      6bd4537b2970c5c6ab765c1860b88aa5      Huawei   
5      6cde2a7fba98a39c2fe69b52a5ffa8c4      Xiaomi   
6      7e7e746f7fa507f102d8edb8fe7bcdb8        Sony   
7      a4c81166ac854f229791ba483399b301       Honor   
8      3864b53a73faa09cb0f83878823a1092      Huawei   
9      abb95da7bd0c87213bb3da80bd18b3fb      Huawei   
10     f8b1e3cc200ebb7cdfb5939429950fe1         lge   
11     f9e15046c20c16e19264585d20f19f33      Xiaomi   
12     947866a16a3361a54c42843ec8376042      Xiaomi   
13     bca3c313ac1c90818f2761ec854eada8        alps   
14     27070a8df98df8aa6c4e58707a1590f4     samsung   
15     1bdab3a0f66515900f3e858c4bc7c7a4      Huawei   
16     63e579bfe2febf44b1f6cbe5cf4a0130     samsung   
17     808

In [8]:
# -*- coding:utf-8 -*-


#######  尝试骚操作，单独针对这个表
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import LinearSVC

train = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_train.tsv', sep='\t', header=None)
test = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_test.tsv', sep='\t', header=None)
def get_label(row):
    if row[1] == 1:
        return row[2]
    else:
        return row[2] + 11
train['label'] = train.apply(lambda row:get_label(row), axis=1)
data_all = pd.concat([train, test], axis=0)
data_all = data_all.rename({0:'id'}, axis=1)
del data_all[1],data_all[2]

deviceid_packages = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_packages.tsv', sep='\t', header=None)
deviceid_packages = deviceid_packages.rename({0: 'id', 1: 'packages_names'}, axis=1)
package_label = pd.read_csv('/Users/xuelili/eguan data/Demo/package_label.tsv', sep='\t', header=None)
package_label = package_label.rename({0:'packages_name', 1:'packages_type'},axis=1)
# package_label['packages_type'] = package_label.apply(lambda row:row['packages_type'] + ' ' + row[2], axis=1)
dict_label = dict(zip(list(package_label['packages_name']), list(package_label['packages_type'])))

data_all = pd.merge(data_all, deviceid_packages, on='id', how='left')

feature = pd.DataFrame()

import numpy as np

# app个数
# 毒特征？
# feature['app_count'] = data_all['packages_names'].apply(lambda row: len(str(row).split(',')))

# 对此数据做countvector,和tfidfvector,并在一起跑几个学习模型
# 引申出来的count和tfidf，跑基本机器学习分类模型
data_all['package_str'] = data_all['packages_names'].apply(lambda row: str(row).replace(',', ' '))
def get_more_information(row):
    result = ' '
    start = True
    row_list = row.split(',')
    for i in row_list:
        try:
            if start:
                result = dict_label[i]
                start = False
            else:
                result = result + ' ' + dict_label[i]
        except KeyError:
            pass
    return result
data_all['package_str_more_information'] = data_all['packages_names'].apply(lambda row: get_more_information(str(row)))

print(data_all)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from sklearn.cross_validation import StratifiedKFold


count_vec = CountVectorizer()
count_csr_basic = count_vec.fit_transform(data_all['package_str'])
tfidf_vec = TfidfVectorizer()
tfidf_vec_basic = tfidf_vec.fit_transform(data_all['package_str'])

count_vec = CountVectorizer()
count_csr_more = count_vec.fit_transform(data_all['package_str_more_information'])

tfidf_vec = TfidfVectorizer()
tfidf_vec_more = tfidf_vec.fit_transform(data_all['package_str_more_information'])

data_feature = scipy.sparse.csr_matrix(scipy.sparse.hstack([count_csr_basic, tfidf_vec_basic,
                     count_csr_more, tfidf_vec_more]))

train_feature = data_feature[:len(train)]
score = train['label']
test_feature = data_feature[len(train):]
number = len(np.unique(score))

# 五则交叉验证
n_folds = 5
print('处理完毕')

########################### lr(LogisticRegression) ################################
print('lr stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    clf = LogisticRegression(random_state=1017, C=8)
    clf.fit(train_feature[tr], score[tr])
    score_va = clf.predict_proba(train_feature[va])
    score_te = clf.predict_proba(test_feature)
    print('得分' + str(mean_squared_error(score[va], clf.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_lr_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_lr_error_single_classfiy.csv', index=None, encoding='utf8')
print('lr特征已保存\n')

########################### SGD(随机梯度下降) ################################
print('sgd stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    sgd = SGDClassifier(random_state=1017, loss='log')
    sgd.fit(train_feature[tr], score[tr])
    score_va = sgd.predict_proba(train_feature[va])
    score_te = sgd.predict_proba(test_feature)
    print('得分' + str(mean_squared_error(score[va], sgd.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_sgd_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_sgd_error_single_classfiy.csv', index=None, encoding='utf8')
print('sgd特征已保存\n')

########################### pac(PassiveAggressiveClassifier) ################################
print('PAC stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    pac = PassiveAggressiveClassifier(random_state=1017)
    pac.fit(train_feature[tr], score[tr])
    score_va = pac._predict_proba_lr(train_feature[va])
    score_te = pac._predict_proba_lr(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], pac.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_pac_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_pac_error_single_classfiy.csv', index=None, encoding='utf8')
print('pac特征已保存\n')


########################### ridge(RidgeClassfiy) ################################
print('RidgeClassfiy stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    ridge = RidgeClassifier(random_state=1017)
    ridge.fit(train_feature[tr], score[tr])
    score_va = ridge._predict_proba_lr(train_feature[va])
    score_te = ridge._predict_proba_lr(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], ridge.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test +=                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      \
        score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_ridge_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_ridge_error_single_classfiy.csv', index=None, encoding='utf8')
print('ridge特征已保存\n')


########################### bnb(BernoulliNB) ################################
print('BernoulliNB stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    bnb = BernoulliNB()
    bnb.fit(train_feature[tr], score[tr])
    score_va = bnb.predict_proba(train_feature[va])
    score_te = bnb.predict_proba(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], bnb.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_bnb_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_bnb_error_single_classfiy.csv', index=None, encoding='utf8')
print('BernoulliNB特征已保存\n')

########################### mnb(MultinomialNB) ################################
print('MultinomialNB stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    mnb = MultinomialNB()
    mnb.fit(train_feature[tr], score[tr])
    score_va = mnb.predict_proba(train_feature[va])
    score_te = mnb.predict_proba(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], mnb.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_mnb_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_mnb_error_single_classfiy.csv', index=None, encoding='utf8')
print('MultinomialNB特征已保存\n')

############################ Linersvc(LinerSVC) ################################
print('LinerSVC stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    lsvc = LinearSVC(random_state=1017)
    lsvc.fit(train_feature[tr], score[tr])
    score_va = lsvc._predict_proba_lr(train_feature[va])
    score_te = lsvc._predict_proba_lr(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], lsvc.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_lsvc_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_lsvc_error_single_classfiy.csv', index=None, encoding='utf8')
print('LSVC特征已保存\n')


kmeans_result = pd.DataFrame()
###### kmeans ###
def get_cluster(num_clusters):
    print('开始' + str(num_clusters))
    name = 'kmean'
    print(name)
    model = KMeans(n_clusters=num_clusters, max_iter=300, n_init=1, \
                        init='k-means++', n_jobs=10, random_state=1017)
    result = model.fit_predict(data_feature)
    kmeans_result[name + 'word_' + str(num_clusters)] = result

get_cluster(5)
get_cluster(10)
get_cluster(19)
get_cluster(30)
get_cluster(40)
get_cluster(50)
get_cluster(60)
get_cluster(70)
kmeans_result.to_csv('/Users/xuelili/eguan data/feature1/cluster_tfidf_feature.csv', index=False)



feature.to_csv('/Users/xuelili/eguan data/feature1/deviceid_package_feature.csv', index=False)


/Users/xuelili/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


                                     id  label  \
0      bd86d59afa24a839ce6029d718accb19    3.0   
1      e7d158c9a8262a35c9cc630a15a9103e    5.0   
2      97abdc3828448b5acc7428dd307bc635   16.0   
3      e4dbdbf07c9cff03d79f4872e65742b4    4.0   
4      6bd4537b2970c5c6ab765c1860b88aa5    3.0   
5      6cde2a7fba98a39c2fe69b52a5ffa8c4    5.0   
6      7e7e746f7fa507f102d8edb8fe7bcdb8   14.0   
7      a4c81166ac854f229791ba483399b301   15.0   
8      3864b53a73faa09cb0f83878823a1092    3.0   
9      abb95da7bd0c87213bb3da80bd18b3fb    5.0   
10     f8b1e3cc200ebb7cdfb5939429950fe1    7.0   
11     f9e15046c20c16e19264585d20f19f33    5.0   
12     947866a16a3361a54c42843ec8376042    5.0   
13     bca3c313ac1c90818f2761ec854eada8    6.0   
14     27070a8df98df8aa6c4e58707a1590f4    6.0   
15     1bdab3a0f66515900f3e858c4bc7c7a4    7.0   
16     63e579bfe2febf44b1f6cbe5cf4a0130   15.0   
17     808ba0f35e382ed557b138e21e5b6af0    6.0   
18     451cfe472891e82634272af946b92b76    6.0   


/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


处理完毕
lr stacking
stack:1/5
得分49.90437649880096
stack:2/5
得分50.15843662534986
stack:3/5
得分49.7383
stack:4/5
得分49.35710713213964
stack:5/5
得分49.27164448003203
lr特征已保存

sgd stacking
stack:1/5


/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


得分44.002298161470826
stack:2/5
得分45.991903238704516
stack:3/5
得分46.8648
stack:4/5
得分52.582674802440735
stack:5/5
得分43.964668201381244
sgd特征已保存

PAC stacking
stack:1/5


/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[[0.00587332 0.03673883 0.00551693 ... 0.00397746 0.01435959 0.00907995]
 [0.00134711 0.01737745 0.00730973 ... 0.00427605 0.04782406 0.02586316]
 [0.0163335  0.04721171 0.03232842 ... 0.01791206 0.0228415  0.10690068]
 ...
 [0.05083309 0.0512308  0.0563345  ... 0.02781791 0.05110699 0.04244682]
 [0.0060436  0.09212574 0.24268415 ... 0.00428525 0.00570819 0.307642  ]
 [0.0492692  0.11155401 0.04791792 ... 0.01003315 0.00445243 0.21538434]]
得分49.730415667466026
stack:2/5
[[0.03022009 0.04605072 0.03843114 ... 0.0240786  0.01839375 0.01586874]
 [0.05528422 0.08056163 0.01932506 ... 0.07806937 0.0368258  0.02010566]
 [0.01045953 0.03854288 0.01264703 ... 0.05113334 0.04118316 0.03606249]
 ...
 [0.01495287 0.03472203 0.01365677 ... 0.06018729 0.0331837  0.14529918]
 [0.00653294 0.01104729 0.01671552 ... 0.08444605 0.03528615 0.23630782]
 [0.03855358 0.1187416  0.01856813 ... 0.04443978 0.04226587 0.0049353 ]]
得分58.33426629348261
stack:3/5
[[0.00942006 0.05245863 0.00435479 ... 0.0690808  0

[[7.92342102e-09 3.77220412e-05 8.37280965e-04 ... 7.93803226e-05
  6.76235684e-06 1.35433276e-04]
 [6.26473998e-15 6.81022590e-07 1.12131784e-03 ... 4.17935375e-05
  5.58769565e-07 1.94627615e-06]
 [3.25736403e-18 1.63020903e-08 1.11398206e-05 ... 8.93037574e-07
  6.85143647e-10 1.22781855e-07]
 ...
 [4.83283842e-14 4.44983164e-08 2.02713211e-05 ... 1.44200223e-04
  1.99303176e-05 3.55537611e-04]
 [1.89614346e-20 3.51766923e-11 1.50977028e-04 ... 6.71812466e-07
  6.33615367e-09 6.00170998e-06]
 [3.10178991e-17 1.41195543e-06 7.40051838e-04 ... 7.37013435e-07
  1.57045432e-09 2.28342227e-09]]
得分45.59606157536985
stack:3/5
[[2.13507352e-17 8.43249771e-08 3.78685517e-04 ... 2.83790213e-05
  6.53075276e-08 1.46347376e-06]
 [5.45010673e-12 3.03322688e-06 4.24069066e-03 ... 1.03563513e-03
  4.87508033e-05 1.16417108e-04]
 [1.03411140e-17 9.40645421e-09 3.06627636e-04 ... 3.92240912e-05
  1.82436569e-07 1.26101191e-05]
 ...
 [3.03399545e-22 1.35896069e-13 7.29706951e-09 ... 2.60469315e-08
  

In [3]:
# -*- coding:utf-8 -*-

import pandas as pd
import scipy.sparse
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

train = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_train.tsv', sep='\t', header=None)
test = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_test.tsv', sep='\t', header=None)

data_all = pd.concat([train, test], axis=0)
data_all = data_all.rename({0:'id'}, axis=1)
del data_all[1],data_all[2]

start_close_time = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_package_start_close.tsv', sep='\t', header=None)
start_close_time = start_close_time.rename({0:'id', 1:'app_name', 2:'start_time', 3:'close_time'}, axis=1)

start_close_time = start_close_time.sort_values(by='start_time')

start_close_time['start_time'] = map(int,start_close_time['start_time']/1000)
start_close_time['close_time'] = map(int,start_close_time['close_time']/1000)

unique_app_name = np.unique(start_close_time['app_name'])
dict_label = dict(zip(list(unique_app_name), list(np.arange(0, len(unique_app_name), 1))))
import time
start_close_time['app_name'] = start_close_time['app_name'].apply(lambda row: str(dict_label[row]))

del start_close_time['start_time'], start_close_time['close_time']

from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
def dealed_row(row):
    app_name_list = list(row['app_name'])
    return ' '.join(app_name_list)

data_feature = start_close_time.groupby('id').progress_apply(lambda row:dealed_row(row)).reset_index()
data_feature = pd.merge(data_all, data_feature, on='id', how='left')
del data_feature['id']

count_vec = CountVectorizer(ngram_range=(1,3))
count_csr_basic = count_vec.fit_transform(data_feature[0])
tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
tfidf_vec_basic = tfidf_vec.fit_transform(data_feature[0])

data_feature = scipy.sparse.csr_matrix(scipy.sparse.hstack([count_csr_basic, tfidf_vec_basic]))


from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.cross_validation import StratifiedKFold

train = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_train.tsv', sep='\t', header=None)
test = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_test.tsv', sep='\t', header=None)
def get_label(row):
    if row[1] == 1:
        return row[2]
    else:
        return row[2] + 11
train['label'] = train.apply(lambda row:get_label(row), axis=1)
data_all = pd.concat([train, test], axis=0)
data_all = data_all.rename({0:'id'}, axis=1)
del data_all[1],data_all[2]

train_feature = data_feature[:len(train)]
score = train['label']
test_feature = data_feature[len(train):]
number = len(np.unique(score))

# 五则交叉验证
n_folds = 5
print('处理完毕')

########################### lr(LogisticRegression) ################################
print('lr stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    clf = LogisticRegression(random_state=1017, C=8)
    clf.fit(train_feature[tr], score[tr])
    score_va = clf.predict_proba(train_feature[va])
    score_te = clf.predict_proba(test_feature)
    print('得分' + str(mean_squared_error(score[va], clf.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_lr_2_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_lr_1_3_error_single_classfiy.csv', index=None, encoding='utf8')
print('lr特征已保存\n')

########################### SGD(随机梯度下降) ################################
print('sgd stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    sgd = SGDClassifier(random_state=1017, loss='log')
    sgd.fit(train_feature[tr], score[tr])
    score_va = sgd.predict_proba(train_feature[va])
    score_te = sgd.predict_proba(test_feature)
    print('得分' + str(mean_squared_error(score[va], sgd.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_2_sgd_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_sgd_1_3_error_single_classfiy.csv', index=None, encoding='utf8')
print('sgd特征已保存\n')

########################### pac(PassiveAggressiveClassifier) ################################
print('PAC stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    pac = PassiveAggressiveClassifier(random_state=1017)
    pac.fit(train_feature[tr], score[tr])
    score_va = pac._predict_proba_lr(train_feature[va])
    score_te = pac._predict_proba_lr(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], pac.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_pac_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_pac_1_3_error_single_classfiy.csv', index=None, encoding='utf8')
print('pac特征已保存\n')


########################### ridge(RidgeClassfiy) ################################
print('RidgeClassfiy stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    ridge = RidgeClassifier(random_state=1017)
    ridge.fit(train_feature[tr], score[tr])
    score_va = ridge._predict_proba_lr(train_feature[va])
    score_te = ridge._predict_proba_lr(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], ridge.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_ridge_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_ridge_1_3_error_single_classfiy.csv', index=None, encoding='utf8')
print('ridge特征已保存\n')


########################### bnb(BernoulliNB) ################################
print('BernoulliNB stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    bnb = BernoulliNB()
    bnb.fit(train_feature[tr], score[tr])
    score_va = bnb.predict_proba(train_feature[va])
    score_te = bnb.predict_proba(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], bnb.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_bnb_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_bnb_1_3_error_single_classfiy.csv', index=None, encoding='utf8')
print('BernoulliNB特征已保存\n')

########################### mnb(MultinomialNB) ################################
print('MultinomialNB stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    mnb = MultinomialNB()
    mnb.fit(train_feature[tr], score[tr])
    score_va = mnb.predict_proba(train_feature[va])
    score_te = mnb.predict_proba(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], mnb.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_mnb_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_mnb_1_3_error_single_classfiy.csv', index=None, encoding='utf8')
print('MultinomialNB特征已保存\n')


0it [00:00, ?it/s]
72728it [00:38, 1888.86it/s]
/Users/xuelili/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


处理完毕
lr stacking
stack:1/5
得分46.5040967226219
stack:2/5
得分51.74280287884846
stack:3/5
得分49.0971
stack:4/5
得分44.97439231769531
stack:5/5
得分49.08807927134421
lr特征已保存

sgd stacking
stack:1/5


/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:347: RuntimeWarning: divide by zero encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:347: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


得分54.500599520383695
stack:2/5
得分55.456417433026786
stack:3/5
得分48.5582
stack:4/5
得分49.59467840352106
stack:5/5
得分50.08918026223601
sgd特征已保存

PAC stacking
stack:1/5


/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[[1.07060148e-21 3.61064567e-16 1.24156686e-09 ... 5.59414907e-11
  4.25183777e-09 1.01296741e-08]
 [7.39931270e-82 4.95367001e-05 3.96624458e-19 ... 1.16792607e-29
  9.22938769e-21 1.35357967e-46]
 [1.25486042e-06 1.06908011e-01 2.73196756e-04 ... 2.13175077e-05
  4.18602354e-04 2.36701293e-04]
 ...
 [1.17475505e-02 1.79976742e-01 2.42316532e-09 ... 7.08125880e-02
  7.65943266e-02 7.67934253e-02]
 [3.77409397e-11 5.19518230e-02 6.66606195e-01 ... 1.98812749e-11
  4.98474426e-13 1.33030661e-12]
 [1.53224811e-12 3.21240364e-04 3.76836674e-09 ... 7.63117789e-10
  3.86166118e-25 8.37114842e-12]]
得分55.732314148681056
stack:2/5
[[9.75740522e-03 5.36516697e-02 9.90151653e-05 ... 2.26488301e-02
  2.52262578e-02 4.39468875e-02]
 [2.81684217e-02 7.70356733e-02 6.66830103e-02 ... 6.49834367e-03
  4.51273500e-02 1.61507249e-02]
 [6.27575445e-10 2.66496072e-05 5.18454157e-04 ... 2.96488830e-26
  8.55264799e-05 1.51740772e-03]
 ...
 [2.76480345e-02 2.88087624e-02 3.80372732e-02 ... 4.42009166e-02
 

得分50.3304
stack:4/5
[[0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 9.90358282e-099 ... 2.96804571e-206
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 ...
 [0.00000000e+000 2.50736797e-232 2.59491603e-190 ... 0.00000000e+000
  0.00000000e+000 2.07004293e-292]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [8.20293428e-023 3.27204031e-011 1.30739222e-005 ... 4.98810791e-012
  1.46919301e-017 2.28613374e-012]]
得分48.79403821146344
stack:5/5
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
得分50.937844059653685
MultinomialNB特征已保存



# 5_get_feature_device_start_close_tfidf

In [4]:
# -*- coding:utf-8 -*-

import pandas as pd
import scipy.sparse
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

train = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_train.tsv', sep='\t', header=None)
test = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_test.tsv', sep='\t', header=None)

data_all = pd.concat([train, test], axis=0)
data_all = data_all.rename({0:'id'}, axis=1)
del data_all[1],data_all[2]

start_close_time = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_package_start_close.tsv', sep='\t', header=None)
start_close_time = start_close_time.rename({0:'id', 1:'app_name', 2:'start_time', 3:'close_time'}, axis=1)

start_close_time['start_time'] = map(int,start_close_time['start_time']/1000)
start_close_time['close_time'] = map(int,start_close_time['close_time']/1000)

unique_app_name = np.unique(start_close_time['app_name'])
dict_label = dict(zip(list(unique_app_name), list(np.arange(0, len(unique_app_name), 1))))
import time
start_close_time['app_name'] = start_close_time['app_name'].apply(lambda row: str(dict_label[row]))

del start_close_time['start_time'], start_close_time['close_time']

from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
def dealed_row(row):
    app_name_list = list(row['app_name'])
    return ' '.join(app_name_list)

data_feature = start_close_time.groupby('id').progress_apply(lambda row:dealed_row(row)).reset_index()
data_feature = pd.merge(data_all, data_feature, on='id', how='left')
del data_feature['id']

count_vec = CountVectorizer()
count_csr_basic = count_vec.fit_transform(data_feature[0])
tfidf_vec = TfidfVectorizer()
tfidf_vec_basic = tfidf_vec.fit_transform(data_feature[0])

data_feature = scipy.sparse.csr_matrix(scipy.sparse.hstack([count_csr_basic, tfidf_vec_basic]))


from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.cross_validation import StratifiedKFold

train = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_train.tsv', sep='\t', header=None)
test = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_test.tsv', sep='\t', header=None)
def get_label(row):
    if row[1] == 1:
        return row[2]
    else:
        return row[2] + 11
train['label'] = train.apply(lambda row:get_label(row), axis=1)
data_all = pd.concat([train, test], axis=0)
data_all = data_all.rename({0:'id'}, axis=1)
del data_all[1],data_all[2]

train_feature = data_feature[:len(train)]
score = train['label']
test_feature = data_feature[len(train):]
number = len(np.unique(score))

# 五则交叉验证
n_folds = 5
print('处理完毕')

########################### lr(LogisticRegression) ################################
print('lr stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    clf = LogisticRegression(random_state=1017, C=8)
    clf.fit(train_feature[tr], score[tr])
    score_va = clf.predict_proba(train_feature[va])
    score_te = clf.predict_proba(test_feature)
    print('得分' + str(mean_squared_error(score[va], clf.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_lr_2_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_lr_2_error_single_classfiy.csv', index=None, encoding='utf8')
print('lr特征已保存\n')

########################### SGD(随机梯度下降) ################################
print('sgd stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    sgd = SGDClassifier(random_state=1017, loss='log')
    sgd.fit(train_feature[tr], score[tr])
    score_va = sgd.predict_proba(train_feature[va])
    score_te = sgd.predict_proba(test_feature)
    print('得分' + str(mean_squared_error(score[va], sgd.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_2_sgd_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_sgd_2_error_single_classfiy.csv', index=None, encoding='utf8')
print('sgd特征已保存\n')

########################### pac(PassiveAggressiveClassifier) ################################
print('PAC stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    pac = PassiveAggressiveClassifier(random_state=1017)
    pac.fit(train_feature[tr], score[tr])
    score_va = pac._predict_proba_lr(train_feature[va])
    score_te = pac._predict_proba_lr(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], pac.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_pac_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_pac_2_error_single_classfiy.csv', index=None, encoding='utf8')
print('pac特征已保存\n')


########################### ridge(RidgeClassfiy) ################################
print('RidgeClassfiy stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    ridge = RidgeClassifier(random_state=1017)
    ridge.fit(train_feature[tr], score[tr])
    score_va = ridge._predict_proba_lr(train_feature[va])
    score_te = ridge._predict_proba_lr(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], ridge.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test +=                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      \
        score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_ridge_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_ridge_2_error_single_classfiy.csv', index=None, encoding='utf8')
print('ridge特征已保存\n')


########################### bnb(BernoulliNB) ################################
print('BernoulliNB stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    bnb = BernoulliNB()
    bnb.fit(train_feature[tr], score[tr])
    score_va = bnb.predict_proba(train_feature[va])
    score_te = bnb.predict_proba(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], bnb.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_bnb_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_bnb_2_error_single_classfiy.csv', index=None, encoding='utf8')
print('BernoulliNB特征已保存\n')

########################### mnb(MultinomialNB) ################################
print('MultinomialNB stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    mnb = MultinomialNB()
    mnb.fit(train_feature[tr], score[tr])
    score_va = mnb.predict_proba(train_feature[va])
    score_te = mnb.predict_proba(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], mnb.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_mnb_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_mnb_2_error_single_classfiy.csv', index=None, encoding='utf8')
print('MultinomialNB特征已保存\n')

############################ Linersvc(LinerSVC) ################################
print('LinerSVC stacking')
stack_train = np.zeros((len(train), number))
stack_test = np.zeros((len(test), number))
score_va = 0

for i, (tr, va) in enumerate(StratifiedKFold(score, n_folds=n_folds, random_state=1017)):
    print('stack:%d/%d' % ((i + 1), n_folds))
    lsvc = LinearSVC(random_state=1017)
    lsvc.fit(train_feature[tr], score[tr])
    score_va = lsvc._predict_proba_lr(train_feature[va])
    score_te = lsvc._predict_proba_lr(test_feature)
    print(score_va)
    print('得分' + str(mean_squared_error(score[va], lsvc.predict(train_feature[va]))))
    stack_train[va] += score_va
    stack_test += score_te
stack_test /= n_folds
stack = np.vstack([stack_train, stack_test])
df_stack = pd.DataFrame()
for i in range(stack.shape[1]):
    df_stack['tfidf_lsvc_classfiy_{}'.format(i)] = np.around(stack[:, i], 6)
df_stack.to_csv('/Users/xuelili/eguan data/feature1/tfidf_lsvc_2_error_single_classfiy.csv', index=None, encoding='utf8')
print('LSVC特征已保存\n')


kmeans_result = pd.DataFrame()
###### kmeans ###
def get_cluster(num_clusters):
    print('开始' + str(num_clusters))
    name = 'kmean'
    print(name)
    model = KMeans(n_clusters=num_clusters, max_iter=300, n_init=1, \
                        init='k-means++', n_jobs=10, random_state=1017)
    result = model.fit_predict(data_feature)
    kmeans_result[name + 'word_' + str(num_clusters)] = result

get_cluster(5)
get_cluster(10)
get_cluster(19)
get_cluster(30)
get_cluster(40)
get_cluster(50)
get_cluster(60)
get_cluster(70)
kmeans_result.to_csv('/Users/xuelili/eguan data/feature1/cluster_2_tfidf_feature.csv', index=False)


0it [00:00, ?it/s]
72728it [00:12, 5621.48it/s] 
/Users/xuelili/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


处理完毕
lr stacking
stack:1/5
得分48.750099920063946
stack:2/5
得分47.38294682127149
stack:3/5
得分50.3979
stack:4/5
得分49.818245473642094
stack:5/5
得分50.22890601541387
lr特征已保存

sgd stacking
stack:1/5


/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:347: RuntimeWarning: divide by zero encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:347: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


得分58.32434052757794
stack:2/5
得分54.70571771291483
stack:3/5
得分49.0857
stack:4/5
得分51.62488746623987
stack:5/5
得分43.970873786407765
sgd特征已保存

PAC stacking
stack:1/5


/Users/xuelili/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[[1.51730458e-19 4.59352928e-15 1.50602451e-08 ... 3.68797142e-20
  2.16593215e-11 1.50211823e-07]
 [1.67964016e-71 2.74548946e-05 6.39193878e-24 ... 2.86127495e-34
  3.57820964e-14 1.65788827e-35]
 [3.02554938e-06 3.80380243e-02 2.75101992e-05 ... 9.76746112e-10
  5.81105853e-06 8.82889954e-05]
 ...
 [4.19144886e-02 8.82787745e-05 1.30262622e-01 ... 1.34972763e-05
  6.39630569e-02 7.76984370e-02]
 [4.21196976e-11 2.47730573e-03 2.14156658e-04 ... 6.04036389e-13
  4.54804585e-15 8.94925505e-14]
 [4.22898567e-03 8.94339115e-03 7.19653666e-03 ... 1.31396482e-11
  2.11174039e-26 5.20629227e-14]]
得分56.64208633093525
stack:2/5
[[2.98516522e-02 6.78941092e-03 4.47653640e-03 ... 1.59544714e-02
  2.45664840e-02 1.96590579e-02]
 [4.18691104e-02 1.07178524e-01 4.52126967e-02 ... 2.34799797e-02
  3.70674100e-02 1.68290210e-02]
 [4.26235620e-05 9.94543550e-02 3.75868009e-11 ... 3.12552477e-13
  9.79015121e-06 7.79838061e-06]
 ...
 [2.72361665e-02 2.91916809e-02 3.10286979e-02 ... 4.41995499e-02
  

得分43.14913422079872
BernoulliNB特征已保存

MultinomialNB stacking
stack:1/5
[[0.00000000e+000 1.60091567e-162 8.91962036e-090 ... 5.77126846e-111
  5.15064153e-119 1.22235163e-026]
 [0.00000000e+000 0.00000000e+000 4.28144753e-179 ... 4.58307872e-112
  7.08369633e-070 1.50797321e-098]
 [1.06635145e-131 9.53018127e-045 2.86056260e-045 ... 2.03900004e-020
  5.83173644e-030 4.60231868e-043]
 ...
 [2.23514972e-015 1.00000000e+000 6.81035396e-011 ... 7.68166339e-061
  1.26943996e-059 4.44900347e-040]
 [6.71604293e-198 1.00000000e+000 1.22474525e-035 ... 3.23979253e-251
  2.85344281e-312 0.00000000e+000]
 [1.34296582e-059 1.00000000e+000 1.71185840e-074 ... 1.00214755e-214
  0.00000000e+000 5.34963473e-210]]
得分53.64718225419664
stack:2/5
[[1.69550126e-034 4.83460686e-034 7.35583927e-038 ... 1.53053307e-038
  9.84690535e-037 3.31260845e-024]
 [1.36477000e-010 9.41685182e-004 2.50449382e-001 ... 3.49223994e-006
  7.61149211e-009 5.03303197e-011]
 [9.94263673e-113 3.97869378e-033 5.41200902e-083 ...

# 6_get_feature_device_start_close

In [5]:
# -*- coding:utf-8 -*-

import pandas as pd
import numpy as np
from sklearn import preprocessing

train = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_train.tsv', sep='\t', header=None)
test = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_test.tsv', sep='\t', header=None)

data_all = pd.concat([train, test], axis=0)
data_all = data_all.rename({0:'id'}, axis=1)
del data_all[1],data_all[2]

start_close_time = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_package_start_close.tsv', sep='\t', header=None)
start_close_time = start_close_time.rename({0:'id', 1:'app_name', 2:'start_time', 3:'close_time'}, axis=1)


start_close_time['diff_time'] = (start_close_time['close_time'] - start_close_time['start_time'])/1000

print('开始转换时间')
import time
start_close_time['close_time'] = start_close_time['close_time'].apply(lambda row: int(time.localtime(row/1000).tm_hour))
start_close_time['start_time'] = start_close_time['start_time'].apply(lambda row: int(time.localtime(row/1000).tm_hour))

# 一个表里面的总次数
print('一个表的总次数')
feature = pd.DataFrame()
feature['start_close_count'] = pd.merge(data_all, start_close_time.groupby('id').size().reset_index(), on='id', how='left')[0]

# 0 - 5 点的使用次数
temp = start_close_time[(start_close_time['close_time'] >=0)&(start_close_time['close_time'] <=5)]
temp = temp.groupby('id').size().reset_index()
feature['zero_five_count'] = pd.merge(data_all, temp, on='id', how='left').fillna(0)[0]

# 玩的时间最长的app的名字编码
def get_max_label(row):
    row_name = list(row['app_name'])
    row_diff_time = list(row['diff_time'])
    return row_name[np.argmax(row_diff_time)]

start_close_max_name = start_close_time.groupby('id').apply(lambda row:get_max_label(row)).reset_index()
label_encoder = preprocessing.LabelEncoder()
feature['start_close_max_name'] = label_encoder.fit_transform(pd.merge(data_all, start_close_max_name, on='id', how='left').fillna(0)[0])

feature.to_csv('/Users/xuelili/eguan data/feature1/feature_start_close.csv', index=False)

开始转换时间
一个表的总次数
